In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import json
import pickle
from collections import defaultdict
%matplotlib inline

In [2]:
np.random.seed(21)

In [3]:
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Activation, Cropping2D, Reshape, BatchNormalization
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.losses import mean_squared_error, categorical_crossentropy
from keras.preprocessing.image import Iterator
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [4]:
from __future__ import absolute_import
from __future__ import print_function

from pkg_resources import parse_version
from keras.callbacks import Callback

In [5]:
class TensorBoardCallBack(Callback):
    """Tensorboard basic visualizations.
    This callback writes a log for TensorBoard, which allows
    you to visualize dynamic graphs of your training and test
    metrics, as well as activation histograms for the different
    layers in your model.
    TensorBoard is a visualization tool provided with TensorFlow.
    If you have installed TensorFlow with pip, you should be able
    to launch TensorBoard from the command line:
    ```
    tensorboard --logdir=/full_path_to_your_logs
    ```
    You can find more information about TensorBoard
    [here](https://www.tensorflow.org/versions/master/how_tos/summaries_and_tensorboard/index.html).
    # Arguments
        log_dir: the path of the directory where to save the log
            files to be parsed by Tensorboard
        batch_freq: frequency (in batch) at which to log data
            If set to 0, we just log at the end of an epoch,
    """

    def __init__(self, log_dir='./logs',
                 batch_freq=0):
        super(TensorBoardCallBack, self).__init__()

        self.log_dir = log_dir
        self.batch_freq = batch_freq
        self.merged = None
        self.writer = tf.summary.FileWriter(self.log_dir)
        self.last_batch = 0
        self.batch_offset = 0

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()

        if hasattr(tf, 'merge_all_summaries'):
            self.merged = tf.merge_all_summaries()
        else:
            self.merged = tf.summary.merge_all()

    def on_batch_end(self, batch, logs=None):
        # Each time we go back to batch 0, we increase the batch_offset
        if batch < self.last_batch:
            self.batch_offset += self.last_batch + 1
        self.last_batch = batch

        batch_cross_epoch = self.batch_offset + batch
        if batch_cross_epoch % self.batch_freq == 0:
            logs = logs or {}

            for name, value in logs.items():
                if name in ['batch', 'size']:
                    continue
                summary = tf.Summary()
                summary_value = summary.value.add()
                summary_value.simple_value = value.item()
                summary_value.tag = name
                self.writer.add_summary(summary, batch_cross_epoch)
            self.writer.flush()

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(summary, epoch)
        self.writer.flush()

    def on_train_end(self, _):
        self.writer.close()

In [6]:
def unet_down_block(x, n_filters, block_id, with_maxpool=True, activation="elu"):
    y = Conv2D(n_filters, (3, 3), activation=activation, 
               padding='valid', name="conv{}_1".format(block_id))(x)
    print(y.shape)
    y = BatchNormalization(name="bn{}_1".format(block_id))(y)
    
    y = Conv2D(n_filters, (3, 3), activation=activation,
               padding='valid', name="conv{}_2".format(block_id))(y)
    conv = BatchNormalization(name="bn{}_2".format(block_id))(y)
    print(y.shape)
    if not with_maxpool:
        return conv
    
    pool = MaxPooling2D(pool_size=(2, 2), name="max_pool{}".format(block_id))(conv)
    print(pool.shape)
    return conv, pool    

In [7]:
def unet_up_block(x, y, n_filters, block_id, activation="elu"):
    up_x = UpSampling2D(size=(2, 2), name="upsample{}".format(block_id))(x)
    
    # Compute crop needed to have the same shape for up_x and y
    _, hx, wx, _ = up_x.shape
    _, hy, wy, _ = y.shape
    cropy = int(hy - hx)//2
    cropx = int(wy - wx)//2
    crop_y = Cropping2D(cropping=((cropy, cropy), (cropx, cropx)),
                        name="crop{}".format(block_id))(y)
    print("Crop: ", cropy, cropx)
    up = concatenate([up_x, crop_y], axis=-1,
                     name="concat{}".format(block_id))
    print(up.shape)
    up = Conv2D(n_filters, (3, 3), 
                activation=activation,
                padding='valid',
                name="conv{}_1".format(block_id))(up)
    print(up.shape)
    up = Conv2D(n_filters, (3, 3),
                activation=activation,
                padding='valid',
                name="conv{}_2".format(block_id))(up)
    print(up.shape)
    return up

In [8]:
max_count = 128
precision = 2
n_bins = max_count/precision + 1

In [9]:
def get_cntnet(im_height, im_width, n_channels=1, n_classes=6,
                 n_filters=[64, 128, 256, 512, 1024]):
    inputs = Input((im_height, im_width, n_channels))
    
    conv1, pool1 = unet_down_block(inputs, n_filters[0], 1)
    conv2, pool2 = unet_down_block(pool1,  n_filters[1], 2)
    conv3, pool3 = unet_down_block(pool2,  n_filters[2], 3)
    conv4, pool4 = unet_down_block(pool3,  n_filters[3], 4)
    conv5 = unet_down_block(pool4, n_filters[4], 5, with_maxpool=False)
    conv6 = Conv2D(64, (1, 1), activation="elu")(conv5)
    flatten = Flatten(name="flat_conv5")(conv6)
    counts = []
    for i in range(5):
        cnt = Dense(64, activation='softmax', name='counts_{}'.format(i))(flatten)
        counts.append(cnt)
    model = Model(inputs=[inputs], outputs=counts, name="cnt_net")
    return model

In [10]:
unet = get_cntnet( 512, 512, 1, n_filters=[32, 64, 128, 256, 512])

(?, 510, 510, 32)
(?, 508, 508, 32)
(?, 254, 254, 32)
(?, 252, 252, 64)
(?, 250, 250, 64)
(?, 125, 125, 64)
(?, 123, 123, 128)
(?, 121, 121, 128)
(?, 60, 60, 128)
(?, 58, 58, 256)
(?, 56, 56, 256)
(?, 28, 28, 256)
(?, 26, 26, 512)
(?, 24, 24, 512)


In [11]:
t = np.ones((1, 512, 512, 1))
counts = unet.predict(t)
print("Counts output: ", counts)

Counts output:  [array([[ 0.01679845,  0.0147111 ,  0.01433711,  0.01361067,  0.01527904,
         0.01796181,  0.01472435,  0.01757992,  0.01453477,  0.01416406,
         0.01636623,  0.01666088,  0.01537805,  0.01648011,  0.01295653,
         0.01746112,  0.01436688,  0.01468341,  0.01658837,  0.01318694,
         0.01410331,  0.01552801,  0.01436455,  0.01290928,  0.01434053,
         0.01201358,  0.0167123 ,  0.01479643,  0.01619869,  0.0126367 ,
         0.01844923,  0.01509077,  0.01334485,  0.01703164,  0.01854808,
         0.01721284,  0.01693775,  0.01912404,  0.0164237 ,  0.01537149,
         0.01498435,  0.01671297,  0.01678836,  0.01816384,  0.01475664,
         0.01754376,  0.01434493,  0.01458366,  0.01415847,  0.01780373,
         0.01689469,  0.01787047,  0.0150187 ,  0.01827472,  0.01561543,
         0.01274641,  0.01526002,  0.0149188 ,  0.0141732 ,  0.01747424,
         0.01684515,  0.01349454,  0.01547644,  0.01712885]], dtype=float32), array([[ 0.01468844,  0.01458

In [12]:
crop = (512 - 24 * 16)//2

In [13]:
crop

64

In [14]:
unet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 512, 512, 1)   0                                            
____________________________________________________________________________________________________
conv1_1 (Conv2D)                 (None, 510, 510, 32)  320                                          
____________________________________________________________________________________________________
bn1_1 (BatchNormalization)       (None, 510, 510, 32)  128                                          
____________________________________________________________________________________________________
conv1_2 (Conv2D)                 (None, 508, 508, 32)  9248                                         
___________________________________________________________________________________________

In [15]:
class NonValidPatch(Exception):
    pass

In [16]:
class PatchIterator(Iterator):
    """Iterator yielding training samples
    :param root_dir: Directory containing training images, density map and sampling map.
    :param image_ids: Set of image ids to use to sample patches.
    :param n_samples_per_image: Number of patches to sample on each image.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 1) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 class_weights,
                 n_samples_per_image=160,
                 target_size=(512, 512),
                 scale = 4,
                 crop = (64, 64),
                 batch_size=16, shuffle=True, seed=42,
                 debug_dir=None):
        self.n_samples_per_block = 8
        self.n_sealion_types = 5
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.debug_dir = debug_dir
        # Normalize to use class_weights as a probability distribution.
        if class_weights:
            self.class_weights = np.asarray(class_weights)/np.sum(class_weights)
        else:
            self.class_weights = np.ones((self.n_sealion_types+1))/(self.n_sealion_types + 1)
        self.n_samples_per_image = n_samples_per_image
        self.target_size = target_size
        self.crop = crop
        self.scale = scale
        self.n_indices = len(self.image_ids) * self.n_samples_per_image
        super(PatchIterator, self).__init__(self.n_indices, batch_size//self.n_samples_per_block, shuffle, seed)
        
    def compute_class_distribution(self, n_batches):
        seg_freqs = defaultdict(int)
        count_freqs = {0: defaultdict(int),
                             1: defaultdict(int),
                             2: defaultdict(int),
                             3: defaultdict(int),
                             4: defaultdict(int)}
        for b in range(n_batches):
            _, [by, bcounts] = self.next()
            by = np.argmax(by, axis=-1)
            ids, freqs = np.unique(by, return_counts=True)
            for i in range(ids.shape[0]):
                seg_freqs[ids[i]] += freqs[i]
            for b in range(bcounts.shape[0]):
                counts = bcounts[b]
                for i in range(counts.shape[0]):
                    count_freqs[i][counts[i]] += 1
        
        return seg_freqs, count_freqs

    def normalize_input(self, x_bgr):
        x = x_bgr.copy()
        return x
    
    def denormalize_input(self, x_normed):
        x = x_normed.copy()
        return x

    def normalize_output(self, y):
        yc = y.copy()
        for i in range(y.shape[0]):
            yc[i] = self.quantify(y[i])
        return to_categorical(yc, 64)
    
    def quantify(self, y):
        if y == 0:
            return 0
        else:
            return min(y//2 + 1, 63)
        
    def dequantify(self, y_normed):
        if y_normed == 0:
            return 0
        else:
            cnt = (y_normed - 1) * 2
            if cnt == 0:
                cnt = 1
            return cnt
        
    def denormalize_output(self, y_normed): 
        yc = y_normed.copy()
        for i in range(y_normed.shape[0]):
            yc[i] = self.dequantifiy(y_normed[i])
        return yc
        
    def random_transform(self, x):
        flips = np.random.randint(0, 2, (3,))
        if flips[0]:
            x = np.rot90(x)
        if flips[1]:
            x = np.flipud(x)
        if flips[2]:
            x = np.fliplr(x)
        return x
                     
    def get_dots_in_patch(self, sx, sy, dots):
        dots_in_patch = [[] for _ in dots]
        for i, ds in enumerate(dots):
            for (x, y) in ds:
                if sx <= x < sx + self.target_size[1] and sy <= y < sy + self.target_size[0]:
                    dots_in_patch[i].append((x - sx, y - sy))
        return dots_in_patch
    
    def build_counts(self, dots):
        h = (self.target_size[0] - 2 * self.crop[0])//self.scale
        w = (self.target_size[1] - 2 * self.crop[1])//self.scale
        counts = np.zeros((5, ), dtype=np.int32)
        for c, ds in enumerate(dots):
            for (x, y) in ds:
                xr = int(round((x - self.crop[1])/self.scale))
                yr = int(round((y - self.crop[0])/self.scale))

                if 0 <= xr < w and 0 <= yr < h:
                    counts[c] += 1
        return counts
          
    def get_weights(self, dots):
        # Set probability to 0 if some sealion type is not in the block
        current_weigths = self.class_weights.copy()
        for i in range(self.n_sealion_types):
            if not dots[i]:
                current_weigths[i] = 0
        current_weigths /= np.sum(current_weigths)
        return current_weigths
    
    def sample(self, shape, dots, image_id):
        margin = self.crop[0] + 30
        max_iterations = self.n_samples_per_block * 5
        
        smap = np.load(os.path.join(self.root_dir, "TrainSegmentationIgnored2x", str(image_id) + ".npz"))["smap"]                                                                          
        n_samples = 0
        bx = np.zeros((self.n_samples_per_block, self.target_size[0], self.target_size[1]))
        bcounts = np.zeros((self.n_samples_per_block, 5))

        current_iteration = 0
        weights = self.get_weights(dots)
        
        # Samples n dots, with some probabilty to get the background only
        while n_samples < self.n_samples_per_block and current_iteration < max_iterations:
            current_iteration += 1
            try:
                # Choose an output class randomly
                output_class = np.random.choice(self.n_sealion_types + 1, size=(1, ), p=weights)[0]
                
                # Background, select randomly a patch in the image (high chance to get only background)
                if output_class == self.n_sealion_types:
                    sx_min = 0
                    sx_max = shape[1] - self.target_size[1]
                    sy_min = 0
                    sy_max = shape[0] - self.target_size[0]
                # Choose a dot randomly in that class
                else:  
                    dot_index = np.random.randint(0, len(dots[output_class]))
                    rx, ry = dots[output_class][dot_index]
                    sx_min = min(max(0, rx - self.target_size[1] + margin), shape[1] - self.target_size[1])
                    sx_max = min(max(0, rx - margin), shape[1] - self.target_size[1])
                    sy_min = min(max(0, ry - self.target_size[0] + margin), shape[0] - self.target_size[1])
                    sy_max = min(max(0, ry - margin), shape[0] - self.target_size[0]) 
                
                    if sx_min >= sx_max:
                        sx_min = min(max(0, rx - self.target_size[1]), shape[1] - self.target_size[1])
                        sx_max = min(max(0, rx), shape[1] - self.target_size[1])
                        
                    if sy_min >= sy_max:
                        sy_min = min(max(0, ry - self.target_size[0]), shape[0] - self.target_size[1])
                        sy_max = min(max(0, ry), shape[0] - self.target_size[0]) 
                    
                if sx_min >= sx_max or sy_min >= sy_max:
                        continue
                        
                # Choose the top-left corner so that the dot selected is in the patch.
                sx = np.random.randint(sx_min, sx_max)
                sy = np.random.randint(sy_min, sy_max)

                dots_in_patch = self.get_dots_in_patch(sx, sy, dots)
                
                smap_patch = smap[sy:sy+self.target_size[0], sx:sx+self.target_size[1]]
                counts = self.build_counts(dots_in_patch)
                smap_patch = self.random_transform(smap_patch)
                bx[n_samples, ...] = smap_patch
                bcounts[n_samples, ...] = counts
                n_samples += 1
            except NonValidPatch:
                continue
                    
        if current_iteration < max_iterations:
            return bx, bcounts
        else:
            print("Error with image ", image_id)
            raise Exception("hoho")
    
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                
        batch_x = np.zeros((current_batch_size * self.n_samples_per_block,
                            self.target_size[0],
                            self.target_size[1]),
                           dtype=K.floatx())
        batch_counts = np.zeros((current_batch_size * self.n_samples_per_block, 5), dtype=np.int32)
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_samples_per_image
            image_id = self.image_ids[index]
            with open(os.path.join(self.root_dir, "TrainDots", str(image_id) + ".pkl"), "rb") as pfile:
                dots = pickle.load(pfile)
            with open(os.path.join(self.root_dir, "TrainShape", str(image_id) + ".pkl"), "rb") as pfile:
                shape = pickle.load(pfile)
            
            # Scale dots coordinates
            dots_resized = []
            for ds in dots:
                ds_resized = []
                for (x, y) in ds:
                    ds_resized.append((x//2, y//2))
                dots_resized.append(ds_resized)
                
            shape = (shape[0]//2, shape[1]//2)
                                      
                
            x, counts = self.sample(shape, dots_resized, image_id)
            batch_x[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = x
            batch_counts[i*self.n_samples_per_block:(i+1)*self.n_samples_per_block, ...] = counts
        
        b, h, w = batch_x.shape
        return batch_x.reshape((b, h, w, 1)), [self.normalize_output(batch_counts[:, 0]),
                                               self.normalize_output(batch_counts[:, 1]),
                                               self.normalize_output(batch_counts[:, 2]),
                                               self.normalize_output(batch_counts[:, 3]),
                                               self.normalize_output(batch_counts[:, 4])]

In [17]:
with open("../data/train.json", "r") as jfile:
    train_ids = json.load(jfile)
train_ids = [int(iid) for iid in train_ids]

with open("../data/val.json", "r") as jfile:
    val_ids = json.load(jfile)
val_ids = [int(iid) for iid in val_ids]

In [18]:
class_weights = [1, 1, 0.2, 0.8, 0.7, 0.01]

In [19]:
trainPatchesGenerator = PatchIterator("/home/ubuntu/sealion/data/", train_ids, class_weights)

In [20]:
valPatchesGenerator = PatchIterator("/home/ubuntu/sealion/data/", val_ids, class_weights)

In [21]:
for batch_x, batch_counts in valPatchesGenerator:
    break

In [22]:
print("X shape: ", batch_x.shape)
print("Counts shape: ", batch_counts[0].shape)

X shape:  (16, 512, 512, 1)
Counts shape:  (16, 64)


In [23]:
counts_pred = unet.predict(batch_x, batch_size=1)

In [25]:
for i in range(5):
    print(np.argmax(counts_pred[i], axis=-1))
    print(np.argmax(batch_counts[i], axis=-1))

[35 43 43 43 43 43 43 43 43 43 43 43 43 34 35 43]
[1 2 1 1 2 1 0 2 2 2 2 2 2 2 2 1]
[45 45 45 57 45 45 45 45 45 45 57 45 45 57 45 45]
[1 1 0 1 1 1 1 2 0 1 1 1 2 1 2 1]
[20 20 42 20 20 20 20 20 20 20 20 20 20 20 22 20]
[6 5 5 6 3 0 0 6 0 0 7 9 9 6 6 0]
[46 46 46 46 46 46 46 46 46 46 46 46 46 46 46 46]
[3 3 3 3 2 0 0 3 0 1 0 0 1 1 1 1]
[53 53 53 53 53 53 53 53 53 53 53 53 53 53 53 53]
[0 0 0 0 0 0 0 0 2 0 5 6 8 8 5 0]


In [ ]:
i = 3
plt.figure(figsize=(6, 6))
plt.imshow(valPatchesGenerator.denormalize_input(batch_x[i,:,:, 0]))
print("True counts: ", batch_counts[i, ...])
print("Pred counts: ", counts_pred[i, ...])

In [ ]:
def to_img(smap):
    colors_rgb = [(1, 7, 179), (178, 12, 178), (5, 41, 79), (177, 54, 26), (26, 171, 43), (0, 0, 0), (255, 255, 255)]
    im = np.zeros(smap.shape + (3, ), dtype=np.uint8)
    for i in range(7):
        im[smap==i, :] = colors_rgb[i]
    return im

In [ ]:
seg_freqs, counts_freqs = trainPatchesGenerator.compute_class_distribution(100)

In [ ]:
from keras.losses import mean_squared_error

In [26]:
!rm -rf /home/ubuntu/data/sealion/data/models/segmentation_debug_counts/*

In [27]:
cb_checkpoint = ModelCheckpoint("/home/ubuntu/data/sealion/data/models/segmentation_debug_counts/ckpt_{epoch:02d}-{val_loss:.2f}.h5")
cb_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto', epsilon=0.01, cooldown=0, min_lr=0)
tensorboard_cb = TensorBoardCallBack(log_dir="/home/ubuntu/data/sealion/data/models/segmentation_debug_counts/log_tb", batch_freq=10)

In [28]:
sgd = SGD(lr=1e-4, momentum=0.9, decay=1e-6, nesterov=True)
unet.compile(optimizer=sgd, loss=[categorical_crossentropy, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy, categorical_crossentropy])

In [29]:
h = unet.fit_generator(trainPatchesGenerator, 1000, epochs=20,
                           verbose=1, callbacks=[cb_checkpoint, cb_reduce_lr, tensorboard_cb],
                           validation_data=valPatchesGenerator, validation_steps=200,
                           class_weight=None,
                           max_q_size=8, workers=4, pickle_safe=False,
                           initial_epoch=0)

Epoch 1/20
1000/1000 [==============================] - 2058s - loss: 8.9102 - counts_0_loss: 1.2292 - counts_1_loss: 1.5294 - counts_2_loss: 2.4306 - counts_3_loss: 2.4136 - counts_4_loss: 1.3074 - val_loss: 29.4470 - val_counts_0_loss: 5.5828 - val_counts_1_loss: 5.3369 - val_counts_2_loss: 8.8155 - val_counts_3_loss: 8.4668 - val_counts_4_loss: 1.2451
Epoch 2/20
1000/1000 [==============================] - 2051s - loss: 7.3916 - counts_0_loss: 0.7860 - counts_1_loss: 1.2302 - counts_2_loss: 2.1305 - counts_3_loss: 2.1500 - counts_4_loss: 1.0949 - val_loss: 42.8922 - val_counts_0_loss: 10.4355 - val_counts_1_loss: 12.7339 - val_counts_2_loss: 9.5958 - val_counts_3_loss: 6.8701 - val_counts_4_loss: 3.2569
Epoch 3/20
 999/1000 [============================>.] - ETA: 1s - loss: 6.9232 - counts_0_loss: 0.6589 - counts_1_loss: 1.0825 - counts_2_loss: 2.0314 - counts_3_loss: 2.0948 - counts_4_loss: 1.0556
Epoch 00002: reducing learning rate to 9.999999747378752e-06.
1000/1000 [============

KeyboardInterrupt: 

In [ ]:
for batch_x, counts in valPatchesGenerator:
    break

In [ ]:
counts_pred = unet.predict(batch_x)

In [ ]:
i = 3
plt.figure(figsize=(6, 6))
plt.imshow(valPatchesGenerator.denormalize_input(batch_x[i,:,:, 0]))
print("True counts: ", batch_counts[i, ...])
print("Pred counts: ", counts_pred[i, ...])

In [ ]:
unet.save("../data/unet_segmentation_ellipse_dmap_sgd_10epochs_200steps.h5")

In [ ]:
for batch_x, batch_y in valPatchesGenerator:
    break

In [ ]:
batch_ypred = unet.predict(batch_x)

In [ ]:
batch_ypred.shape

In [ ]:
np.min(batch_ypred[:,:,2])

In [ ]:
gg = np.argmax(batch_ypred, axis=-1)

In [ ]:
np.unique(gg, return_counts=True)

In [ ]:
i = 0
plt.figure(figsize=(15, 5))
plt.subplot(131)
plt.imshow(valPatchesGenerator.denormalize_input(batch_x[i,:,:, :]))
plt.subplot(132)
plt.imshow(np.argmax(batch_y[i,...], axis=-1))
plt.subplot(133)
plt.imshow(np.argmax(batch_ypred[i,...], axis=-1))

In [ ]:
plt.imshow(valPatchesGenerator.denormalize_output(batch_ypred[i,:,:, 0]) > 0.0007)

In [ ]:
i = 7
print("GT: ", np.sum(valPatchesGenerator.denormalize_output(batch_y[i,:,:, 0])))
print("Pred: ", np.sum(valPatchesGenerator.denormalize_output(batch_ypred[i,:,:, 0])))

In [ ]:
def full_image_process(im, net, patchGenerator, patch_size=(224, 224), batch_size=8, overlap=(64, 64)):
    h, w, c = im.shape
    n_patches_x = int(np.ceil((w - patch_size[1])/(patch_size[1] - overlap[1]) + 1))
    n_patches_y = int(np.ceil((h - patch_size[0])/(patch_size[0] - overlap[0]) + 1))
    print(n_patches_x, n_patches_x)
    
    dmap = np.zeros((h, w, 1), dtype=np.float32)
    dmap_count = np.zeros((h, w, 1), dtype=np.int8)
    batch_x = np.zeros((batch_size, ) + patch_size + (c, ), dtype=np.float32)
    batch_pos = np.zeros((batch_size, 4), dtype=np.int32)
    
    current_batch_size = 0
    for py in range(n_patches_y):
        y_start = py * (patch_size[0] - overlap[0])
        y_start = min(h - patch_size[0], y_start)
        y_end = y_start + patch_size[0]
        for px in range(n_patches_x):
            x_start = px * (patch_size[1] - overlap[1])
            x_start = min(w - patch_size[1], x_start)
            x_end = x_start + patch_size[1]
            
            # Keep filling the batch
            batch_x[current_batch_size, :, :, :] = im[y_start:y_end, x_start:x_end, :]
            batch_pos[current_batch_size, :] = np.array([y_start, y_end, x_start, x_end])
            current_batch_size += 1
            
            if current_batch_size == batch_size or (py == n_patches_y - 1 and px == n_patches_x - 1) :
                # time to predict
                batch_x_normed = patchGenerator.normalize_input(batch_x)
                batch_ylog = net.predict(batch_x_normed)
                batch_y = patchGenerator.denormalize_output(batch_ylog)
                # Fill the full dmap
                for i in range(current_batch_size):
                    y_start, y_end, x_start, x_end = tuple(batch_pos[i,:])
                    dmap[y_start:y_end, x_start:x_end, :] += batch_y[i,:,:,:]
                    dmap_count[y_start:y_end, x_start:x_end] += 1
                current_batch_size = 0
                
    return dmap, dmap_count

In [ ]:
im = cv2.imread("../data/sealion/Train/872.jpg")

In [ ]:
dmap, dmap_count = full_image_process(im, unet, valPatchesGenerator)

In [ ]:
plt.imshow(dmap[:,:,0])

In [ ]:
dmap_avg = dmap/dmap_count

In [ ]:
dmap_count.shape

In [ ]:
np.sum(dmap)

In [ ]:
np.sum(dmap_avg)

In [ ]:
dmap_gt = np.load("../data/sealion/TrainDensity/872_0.npz")